In [1]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas as pd
import geopandas as gpd

In [3]:
# Testing if tutorial works
graph = Graph.from_file("./data/pa/PA.shp")

AttributeError: 'Polygon' object has no attribute 'id'

# Creating the dataframe
First, we create a dataframe including all the information we need: the location of each block, the population of each block, and the district assignment of each block. We also include the total population of each district. The shapefile uses the 2020 census block shapes, and the population data is from the 2020 census too. The block equivalency file is from the North Carolina Legislature's website.

In [2]:
blocks_shp = gpd.read_file("./data/shapefiles/tl_2020_37_tabblock20.shp")
blocks_data = pd.read_csv("./data/nc/DECENNIALPL2020.P1-Data-block.csv")
blocks_data = blocks_data[blocks_data["NAME"].str.contains("Census Tract")]
blocks_data = blocks_data[["GEO_ID", "NAME", "P1_001N", "P1_003N"]] # P1_001N = Total Population, P1_003N = White alone
blocks_data["P1_001N"] = blocks_data["P1_001N"].astype(int)
blocks_data["P1_003N"] = blocks_data["P1_003N"].astype(int)
blocks_data["GEOID"] = blocks_data["GEO_ID"].str[9:]
blocks = blocks_shp.merge(blocks_data, left_on="GEOID20", right_on="GEOID", validate="one_to_one")

C:\Users\patri\AppData\Local\Temp\ipykernel_16324\3237898443.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143) have mixed types. Specify dtype option on import or set low_memory=False.
  blocks_data = pd.read_csv("./data/nc/DECENNIALPL2020.P1-Data-block.csv")


In [3]:
districts = dict(pd.read_csv("./data/nc/equivalency.csv", dtype=str).values)

In [4]:
districts

{'371139706002016': '11',
 '371139706002042': '11',
 '371139706002017': '11',
 '371139706002059': '11',
 '370399302002040': '11',
 '370399302002017': '11',
 '370399302002020': '11',
 '370399302002022': '11',
 '370399302002005': '11',
 '370399302001055': '11',
 '370399302001059': '11',
 '370399302001050': '11',
 '370399302001042': '11',
 '370399302001054': '11',
 '370399302001051': '11',
 '370399302002001': '11',
 '370399302002019': '11',
 '370399302002021': '11',
 '370399302002004': '11',
 '370399302001049': '11',
 '370399302002002': '11',
 '370399302001053': '11',
 '370399302002030': '11',
 '370399302002032': '11',
 '370399302002003': '11',
 '370399302002000': '11',
 '370399301013027': '11',
 '370399301011007': '11',
 '370399301013050': '11',
 '370399302002033': '11',
 '370399302002031': '11',
 '370399302002045': '11',
 '370399302002044': '11',
 '370399301011013': '11',
 '370399301011009': '11',
 '370399301011005': '11',
 '370399301011010': '11',
 '370399301011012': '11',
 '3703993010

In [5]:
blocks["District"] = blocks["GEOID20"].map(districts)

In [6]:
blocks.head()

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,INTPTLON20,HOUSING20,POP20,geometry,GEO_ID,NAME,P1_001N,P1_003N,GEOID,District
0,37,085,070904,1016,370850709041016,Block 1016,G5040,R,NaN,NaN,...,-078.7429098,135,236,"POLYGON ((-78.74612 35.51781, -78.74593 35.517...",1000000US370850709041016,"Block 1016, Block Group 1, Census Tract 709.04...",236,67,370850709041016,13
1,37,105,030300,2040,371050303002040,Block 2040,G5040,U,78877,U,...,-079.1609933,0,0,"POLYGON ((-79.16192 35.46852, -79.16154 35.469...",1000000US371050303002040,"Block 2040, Block Group 2, Census Tract 303, L...",0,0,371050303002040,9
2,37,085,070200,3007,370850702003007,Block 3007,G5040,U,25039,U,...,-078.6080862,6,14,"POLYGON ((-78.60900 35.30999, -78.60828 35.310...",1000000US370850702003007,"Block 3007, Block Group 3, Census Tract 702, H...",14,0,370850702003007,13
3,37,085,070500,3022,370850705003022,Block 3022,G5040,U,25039,U,...,-078.6570878,26,45,"POLYGON ((-78.65879 35.31721, -78.65876 35.317...",1000000US370850705003022,"Block 3022, Block Group 3, Census Tract 705, H...",45,26,370850705003022,13
4,37,085,070401,1041,370850704011041,Block 1041,G5040,U,25039,U,...,-078.6628098,16,46,"POLYGON ((-78.66407 35.35466, -78.66371 35.355...",1000000US370850704011041,"Block 1041, Block Group 1, Census Tract 704.01...",46,9,370850704011041,13


In [7]:
blocks_keep = blocks[['GEOID20', 'geometry', 'District', 'P1_001N', 'P1_003N']]

In [8]:
blocks_keep.head()

,GEOID20,geometry,District,P1_001N,P1_003N
0,370850709041016,"POLYGON ((-78.74612 35.51781, -78.74593 35.517...",13,236,67
1,371050303002040,"POLYGON ((-79.16192 35.46852, -79.16154 35.469...",9,0,0
2,370850702003007,"POLYGON ((-78.60900 35.30999, -78.60828 35.310...",13,14,0
3,370850705003022,"POLYGON ((-78.65879 35.31721, -78.65876 35.317...",13,45,26
4,370850704011041,"POLYGON ((-78.66407 35.35466, -78.66371 35.355...",13,46,9


In [9]:
blocks_keep.isna().sum()

GEOID20     0
geometry    0
District    0
P1_001N     0
P1_003N     0
dtype: int64

# Saving the dataframe as a graph
Next, we convert the dataframe into a graph useing the gerrychain package.

In [11]:
dual_graph = Graph.from_geodataframe(blocks_keep, ignore_errors=True)

AttributeError: 'Polygon' object has no attribute 'id'